In [1]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import time

import numpy as np
import pandas as pd
import math
from Config import *


class node:
    def __int__(self):
        self.items = []
        self.key = 0
        self.sup = 0

    def show(self):
        print(self.items, bin(self.key), end=" ")
        try:
            print(self.sup)
        except:
            print()
            pass


def ListToNode(items):
    nodeT = node()
    nodeT.key = 0
    nodeT.items = items
    for index, i in enumerate(defaultSL):  # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index - 1)
    return nodeT


def keyToList(key):
    ans = []
    global defaultSL
    for i in range(1, len(defaultSL) + 1):
        if key & (1 << (len(defaultSL)) - i) == (1 << (len(defaultSL)) - i):  # 含有第i个
            ans.append(defaultSL[i - 1])
    return ans


# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))

    for i in Ck:
        i.sup = 0
        for item in D:
            if i.key & item.key == i.key:  # 与运算判断子集
                i.sup += 1
    ans = []
    length = len(D)
    for i in Ck:
        i.sup = i.sup / length
        if i.sup >= minSupport:
            ans.append(i)
            # i.show()
    return ans


def aprioriGen(Lk, k, L):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''

    #     print(k)
    left = []  # 存储积最小的两组
    right = []
    lastl = None  # 存储积次小的两组
    lastr = None
    minV = 999999999
    minV2 = 999999999
    ans = []
    nowSet = set()
    for i in range(1, math.ceil(k / 2) + 1):
        A = L[i - 1]
        B = L[k - i - 1]
        #         print(len(A[0].items),len(B[0].items) )
        # print(len(A), len(B))
        if len(A) * len(B) < minV:  # 如果比最小的更小,更新最小的,且更新次小的
            lastl = left
            lastr = right
            left = A
            right = B
            minV2 = minV
            minV = len(A) * len(B)
        elif len(A) * len(B) < minV2:  # 如果没有比更小的小，但是比次小的小，那么更新次小的
            lastl = A
            lastr = B
            minV2 = len(A) * len(B)

    for i in left:
        
        for j in right:
            if i.key & j.key == 0:  # 这俩频繁集合不相交
                t = i.key | j.key
                if t not in nowSet:
                    nowSet.add(t)
                    nodeT = node()
                    nodeT.key = t
                    #                         nodeT.items = keyToList(nodeT.key)  #生成新的items
                    ans.append(nodeT)

            # 在另一组分布里面求可能的组合，然后两个组合取交集， 目前没有好效果
#     returnList = []
#     nowSet2 = set()
#     if k > 3:
#         # print(lastl)
#         left = lastl
#         right = lastr
#         for i in left:
#             for j in right:
#                 if i.key & j.key == 0:  # 这俩频繁集合不相交
#                     t = i.key | j.key
#                     # print(bin(t))
#                     if t not in nowSet2 and t in nowSet :
#                         nowSet2.add(t)
#                         nodeT = node()
#                         nodeT.key = t
#                         #                         nodeT.items = keyToList(nodeT.key)  #生成新的items
#                         returnList.append(nodeT)
#         return returnList
    return ans


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    # C1 = createC1(dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    global defaultSL
    global defaultS

    #     print(defaultSL)
    C1 = [ListToNode([i]) for i in defaultSL]

    # D = [set(var) for var in dataSet]  # 集合化数据集

    F1 = scanD(dataSet, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素

    # print()
    L = [F1]
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t = time.time()
        Ck = aprioriGen(L[k - 2], k, L)  # 计算候选集
        
        TmpCk = scanD(dataSet[0:100], Ck, minSupport *(2/3)) # 使用采样法，进行预处理，减少候选集数目，适当降低minSiupport提高容错度。同时也会降低过滤效果
        
        print(f'gen coast:{time.time() - t:.8f}s')
        t = time.time()

        Fk = scanD(dataSet, TmpCk, minSupport)  # 筛选最小支持度的频繁项集
        print(f'scan coast:{time.time() - t:.8f}s')

        L.append(Fk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList






In [2]:
defaultS = set()  # 存储所有单项的集合
myDat = loadTest()  # 导入数据集
for i in myDat:
    for j in i:
        if j not in defaultS:
            defaultS.add(j)
defaultSL = list(defaultS)
defaultSL = sorted(defaultSL) # 把所有单项计算出来并排序，形成默认顺序，方便后面进行二进制编码
print(defaultSL, len(defaultSL))
Items = []



#对项集进行二进制编码
for items in myDat:
    nodeT = node()
    nodeT.key =0
    nodeT.items = items
    for index,i in enumerate(defaultSL): # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index -1)
    Items.append(nodeT)
    # print(items, bin(nodeT.key))


['m0e', 'm0p', 'm10e', 'm10t', 'm11b', 'm11c', 'm11e', 'm11r', 'm12f', 'm12k', 'm12s', 'm13f', 'm13k', 'm13s', 'm13y', 'm14b', 'm14e', 'm14g', 'm14n', 'm14p', 'm14w', 'm15b', 'm15e', 'm15g', 'm15n', 'm15p', 'm15w', 'm15y', 'm16p', 'm17w', 'm18o', 'm18t', 'm19e', 'm19f', 'm19l', 'm19p', 'm1b', 'm1f', 'm1k', 'm1s', 'm1x', 'm20h', 'm20w', 'm2f', 'm2s', 'm2y', 'm3b', 'm3c', 'm3e', 'm3g', 'm3n', 'm3p', 'm3r', 'm3u', 'm3w', 'm3y', 'm4f', 'm4t', 'm5a', 'm5c', 'm5f', 'm5l', 'm5n', 'm5p', 'm5s', 'm5y', 'm6f', 'm7c', 'm7w', 'm8b', 'm8n', 'm9b', 'm9e', 'm9g', 'm9h', 'm9k', 'm9n', 'm9p', 'm9u', 'm9w'] 80


In [3]:

twoDif =set() # 计算仅仅两个不同时的二进制集合
length = len(defaultSL)
for i in range(length):
    for j in range(length):
        if i != j:
            t=0
#             print(i,j)
            t = 1<<(i) 
            t+=1<<(j)
#             print(bin(t))
            twoDif.add(t)
# for i in twoDif:
#     print(bin(i))


In [4]:
len(Items)

5000

In [5]:
import time
t = time.time()

L = apriori(Items, 0.2)  # 选择频繁项集
# print(u"频繁项集L：")
# for li in L:
#     for i in li:
#         i.show()
print(f'total coast:{time.time() - t:.8f}s')



80
iter is  2
465
gen coast:0.00609684s
214
scan coast:0.14570212s
iter is  3
2992
gen coast:0.03718400s
1040
scan coast:0.66006804s
iter is  4
11348
gen coast:0.13734293s
3201
scan coast:1.94520092s
iter is  5
27410
gen coast:0.34946609s
6729
scan coast:3.98783088s
iter is  6
45402
gen coast:0.56266713s
10072
scan coast:5.85236311s
iter is  7
53830
gen coast:0.66104794s
10974
scan coast:6.30323720s
iter is  8
46642
gen coast:0.55082798s
8768
scan coast:5.00413084s
iter is  9
29572
gen coast:0.36832428s
5103
scan coast:2.88729095s
iter is  10
13441
gen coast:0.15619111s
2115
scan coast:1.19239473s
iter is  11
4163
gen coast:0.04980111s
597
scan coast:0.33909988s
iter is  12
790
gen coast:0.00973916s
105
scan coast:0.05878305s
iter is  13
70
gen coast:0.00088882s
9
scan coast:0.00488281s
total coast:31.32392216s


In [6]:
for i in L:
    print(len(i))

31
174
661
1578
2582
3034
2618
1661
758
236
45
4
0


In [7]:
# 加入多进程进行测试

In [8]:
from Config import *
import ray
ray.init()

RayContext(dashboard_url='', python_version='3.7.13', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-02_01-23-50_774026_4795/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-02_01-23-50_774026_4795/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-05-02_01-23-50_774026_4795', 'metrics_export_port': 65338, 'gcs_address': '127.0.0.1:63835', 'address': '127.0.0.1:63835', 'node_id': '5f0cd9f9a48d46932c43837790baef997387a67a67e2d15b9f14aee0'})

In [14]:
import copy
myDat = loadTest()  # 导入数据集


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    
    print(len(Ck))

    for i in Ck:
        i.sup = 0
        for item in D:
            if i.key & item.key == i.key:  # 与运算判断子集
                i.sup += 1
    ans = []
    length = len(D)
    for i in Ck:
        i.sup = i.sup / length
        if i.sup >= minSupport:
            ans.append(i)
            # i.show()
    return ans


def total_scan(D, Ck, minSupport):
    global Ds
    # 分为10核
    Cs = []
    Ck =list(Ck)
    print(len(Ck))
    single_size = int(len(Ck)/10)
    for i in range(10):
        Cs.append(Ck[i* single_size : (i+1)*single_size])
    Cs.append(Ck[10 * single_size: -1])
    
    t = time.time()
    ans1 = [single_scan.remote(D, Ci, minSupport) for Ci in Cs]
    
    
    ans2 = ray.get(ans1)
    print(f'scan coast:{time.time() - t:.8f}s')

    tmp = {}
    ret_list =[]
    for i in ans2:
        for j in i:
            if j.key not in tmp:
                tmp[j.key]=1
                ret_list.append(j)
    return ret_list
    

@ray.remote
def single_scan(D, Ck, minS):
    c = scanD(D, Ck,minS)
    return c


def aprioriGen(Lk, k, L):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''

    #     print(k)
    left = []  # 存储积最小的两组
    right = []
    lastl = None  # 存储积次小的两组
    lastr = None
    minV = 999999999
    minV2 = 999999999
    ans = []
    nowSet = set()
    for i in range(1, math.ceil(k / 2) + 1):
        A = L[i - 1]
        B = L[k - i - 1]
        #         print(len(A[0].items),len(B[0].items) )
        # print(len(A), len(B))
        if len(A) * len(B) < minV:  # 如果比最小的更小,更新最小的,且更新次小的
            lastl = left
            lastr = right
            left = A
            right = B
            minV2 = minV
            minV = len(A) * len(B)
        elif len(A) * len(B) < minV2:  # 如果没有比更小的小，但是比次小的小，那么更新次小的
            lastl = A
            lastr = B
            minV2 = len(A) * len(B)

    for i in left:
        
        for j in right:
            if i.key & j.key == 0:  # 这俩频繁集合不相交
                t = i.key | j.key
                if t not in nowSet:
                    nowSet.add(t)
                    nodeT = node()
                    nodeT.key = t
                    #                         nodeT.items = keyToList(nodeT.key)  #生成新的items
                    ans.append(nodeT)
    return ans

def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    # C1 = createC1(dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    global defaultSL
    global defaultS

    #     print(defaultSL)
    C1 = [ListToNode([i]) for i in defaultSL]

    # D = [set(var) for var in dataSet]  # 集合化数据集

    F1 = scanD(dataSet, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素

    # print()
    L = [F1]
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t = time.time()
        Ck = aprioriGen(L[k - 2], k, L)  # 计算候选集
        
        TmpCk = scanD(dataSet[0:100], Ck, minSupport *(2/3)) # 使用采样法，进行预处理，减少候选集数目，适当降低minSiupport提高容错度。同时也会降低过滤效果
        
        print(f'gen coast:{time.time() - t:.8f}s')
        t = time.time()

        Fk = total_scan(dataSet, TmpCk, minSupport) # 筛选最小支持度的频繁项集
        print(f'scan coast:{time.time() - t:.8f}s')

        L.append(Fk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息



In [15]:
import time
t = time.time()

L = apriori(Items, 0.2)  # 选择频繁项集
# print(u"频繁项集L：")
# for li in L:
#     for i in li:
#         i.show()
print(f'total coast:{time.time() - t:.8f}s')


80
iter is  2
465
gen coast:0.00643897s
214
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
(single_scan pid=4825) 21
scan coast:0.52720833s(single_scan pid=4825) 21

scan coast:0.52778006s
iter is  3
2978
(single_scan pid=4825) 3
gen coast:0.03709793s
1040
(single_scan pid=4825) 104
(single_scan pid=4818) 104
(single_scan pid=4822) 104
(single_scan pid=4818) 104
(single_scan pid=4822) 104
(single_scan pid=4825) 104
(single_scan pid=4822) 104
(single_scan pid=4825) 104
(single_scan pid=4818) 104
(single_scan pid=4822) 104
scan coast:0.54272699s
scan coast:0.54315019s
iter is  4
11348
(single_scan pid=4818) 0
gen coast:0.17207599s
3201
(single_scan pid=4822) 320
(single_scan pid=4825) 320
(single_scan pid=4818) 320
(single_scan pid=4819) 320
(single_scan pid=4826) 320
(single_scan pid=4821) 320
(single_scan pid=4825) 320
(single_

In [16]:
for i in L:
    print(len(i))

31
173
661
1577
2582
3034
2618
1661
758
236
45
4
0
